Deps

In [14]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [15]:
!pip install transformers torchdata datasets evaluate rouge_score loralib peft

In [16]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

In [17]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [18]:
model_name = 'google/flan-t5-small'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Let's explore the model parameters

In [19]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 76961152
all model parameters: 76961152
percentage of trainable model parameters: 100.00%


In [20]:
index = 222

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

#Person1#: What are you working on?
#Person2#: I'm figuring out my budget.
#Person1#: What budget?
#Person2#: I'm making a shopping budget, so that I don't spend too much money.
#Person1#: How much money can you spend?
#Person2#: I can only spend $ 300 a month.
#Person1#: Why only $ 300?
#Person2#: I need to save the rest.
#Person1#: For what?
#Person2#: I need to pay my bills.
#Person1#: Your budget is a good idea.
#Person2#: I know. It's going to save me a lot of money, I hope.

Summary:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person2# is making a shopping budget to save the rest of the money. #Person1# thinks it's a good idea.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZE

Time to Fine tune, first create the prompt-response pairs.

In [21]:
def tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
    # JS: const prompt = dialogueList.map(dialogue => startPrompt + dialogue + endPrompt);
    example['prompt'] = prompt
    # Why padding is Max Length?
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    # Element's 'input_ids' and 'label' keys have the tokenised prompt and response.
    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)
# print(tokenized_datasets['train'][200])
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary', 'prompt'])
print(tokenized_datasets['train'][200])

# 
# print(tokenized_datasets)

{'input_ids': [12198, 1635, 1737, 8, 826, 3634, 5, 1713, 345, 13515, 536, 4663, 10, 363, 103, 25, 241, 12, 214, 81, 140, 58, 1713, 345, 13515, 357, 4663, 10, 571, 81, 39, 2705, 3187, 44, 1900, 58, 1713, 345, 13515, 536, 4663, 10, 37, 1348, 2769, 13, 66, 82, 2996, 19, 756, 11989, 5, 1713, 345, 13515, 357, 4663, 10, 86, 84, 1426, 410, 25, 129, 8, 2030, 6784, 58, 1713, 345, 13515, 536, 4663, 10, 86, 17082, 27, 530, 3, 9, 3, 3916, 5, 1713, 345, 13515, 357, 4663, 10, 2114, 25, 1204, 136, 19397, 58, 1713, 345, 13515, 536, 4663, 10, 2163, 6, 27, 43, 6, 11, 386, 648, 16, 792, 5, 1713, 345, 13515, 357, 4663, 10, 2114, 25, 118, 3, 9, 853, 2488, 58, 1713, 345, 13515, 536, 4663, 10, 27, 43, 118, 3, 9, 853, 26864, 7, 1208, 16, 1567, 13, 2116, 21, 192, 203, 5, 1713, 345, 13515, 357, 4663, 10, 3963, 25, 1715, 16, 136, 1886, 1087, 58, 1713, 345, 13515, 536, 4663, 10, 27, 47, 46, 30277, 7, 372, 1144, 16, 1900, 5, 1713, 345, 13515, 357, 4663, 10, 363, 2600, 33, 25, 207, 44, 58, 1713, 345, 13515, 536, 46

In [22]:
output_dir = './full-dialogue-summary-training'

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=10,  # Increase the number of epochs for more training.
    weight_decay=0.01,
    logging_steps=200,   # Reduce the frequency of logging for better readability.
)


In [23]:
print(torch.cuda.is_available())

import os
import nltk
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # decode preds and labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # rougeLSum expects newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return result

True


In [24]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

# tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

Shapes of the datasets:
Training: (12460, 2)
Validation: (500, 2)
Test: (1500, 2)


In [25]:
trainer = Trainer(
    model=original_model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)



In [26]:
trainer.train()


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


TrainOutput(global_step=15580, training_loss=42.07952503209243, metrics={'train_runtime': 23397.0484, 'train_samples_per_second': 5.325, 'train_steps_per_second': 0.666, 'total_flos': 2.31619568861184e+16, 'train_loss': 42.07952503209243, 'epoch': 10.0})

In [30]:
index = 200

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

instruct_model = AutoModelForSeq2SeqLM.from_pretrained("./full-dialogue-summary-training/checkpoint-15500", torch_dtype=torch.bfloat16)


prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"
print(device)
original_model.to(device)
instruct_model.to(device)

tokenizer = AutoTokenizer.from_pretrained(model_name)

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1)).to(device)
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)



print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{original_model_text_output}')
print(dash_line)
print(f'MODEL GENERATION - FINE TUNED:\n{instruct_model_text_output}')

cpu
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.

Summary:

---------------------------------------------------------------